In [2]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier, callback
import mlflow


In [3]:


import pandas as pd
from sklearn.metrics import roc_auc_score
from Config import config
import warnings
warnings.filterwarnings("ignore")
import optuna  # pip install optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
import gc
import pickle
import tempfile

In [4]:
from lightgbm.basic import LightGBMError
from lightgbm import LGBMClassifier, early_stopping, log_evaluation, Booster
from sklearn.metrics import roc_auc_score
import os
from catboost import CatBoostClassifier


# Loading data and group split

In [ ]:
# df_pairs = pd.read_csv("/workspace/Training_data/training/Full_training_data.csv")
# df_pairs = df_pairs.sample(frac=1).reset_index(drop=True)
# kf = GroupKFold(n_splits=5)
# for i, (trn_idx, val_idx) in enumerate(kf.split(df_pairs, df_pairs["placeId1"], df_pairs["placeId1"])):
#     df_pairs.loc[val_idx, "fold"] = i
# df_pairs["fold"].value_counts()
# df_pairs.to_csv("/workspace/Training_data/training/Full_training_data_fold.csv",index=None)

# Loading data

In [5]:
df_pairs = pd.read_csv("/workspace/Traininf_model_v4/Training_data/Full_training_data_fold_v4_without_categories.csv")

In [6]:
features = ['similarity', 'haversine', 'name_davies',
       'name_leven', 'name_dice', 'name_jaro', 'name_set_ratio',
       'street_davies', 'street_leven', 'street_jaro', 'email_davies',
       'email_leven', 'email_jaro', 'url_davies', 'url_leven', 'url_jaro',
       'brands_davies', 'brand_leven', 'brand_jaro', 'phone_lcs',
       'subcat_WRatio', 'subcat_ratio', 'subcat_token_set_ratio',
       'Is_direction_match_0', 'Is_direction_match_1', 'Is_direction_match_2',
       'Is_house_match_0', 'Is_house_match_1', 'Is_house_match_2',
       'Is_category_match_0', 'Is_category_match_1', 'Is_subcategory_match_0',
       'Is_subcategory_match_1', 'Is_subcategory_match_2', 'Is_brand_match_0',
       'Is_brand_match_1', 'Is_brand_match_2', 'Is_brand_match_3',
       'Is_related_cat_0', 'Is_related_cat_1', 'Is_name_number_match_0',
       'Is_name_number_match_1', 'Is_name_number_match_2',
       'Is_name_number_match_3', 'is_phone_match_1', 'is_phone_match_2',
       'is_phone_match_3', 'is_phone_match_4', 'Is_email_match_0',
       'Is_email_match_1', 'Is_email_match_2', 'Is_url_match_0',
       'Is_url_match_1', 'Is_url_match_2']

In [7]:
def objective(trial, df_pairs):
    catboost_params = dict(
        depth=trial.suggest_int("depth", 5, 15),
        border_count=trial.suggest_int("border_count", 16, 256),
        min_data_in_leaf=trial.suggest_int("min_data_in_leaf", 100, 10000),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-3, 1, log=True)
    )
    
    cv_scores = np.empty(5)
    cv_scores_train = np.empty(5)
    for fold in [0, 1, 2, 3, 4]:
        X_train = df_pairs[df_pairs["fold"] != fold]
        X_test = df_pairs[df_pairs["fold"] == fold]

        model = CatBoostClassifier(
            **catboost_params,
            learning_rate=0.1,
            n_estimators=10000,
            objective="Logloss",
            eval_metric="AUC",
            gpu_ram_part=0.90,
            task_type="GPU",
            use_best_model = True
        )
        
        model.fit(X_train[features],
                  X_train["duplicate_flag"],verbose=100,
                  eval_set=[(X_test[features], X_test["duplicate_flag"])],
                  early_stopping_rounds=40)

        preds = model.predict_proba(X_test[features])[:, 1]
        preds_train = model.predict_proba(X_train[features])[:, 1]
        cv_scores[fold] = roc_auc_score(X_test["duplicate_flag"], preds)
        cv_scores_train[fold] = roc_auc_score(X_train["duplicate_flag"],
                                              preds_train)
    print(f"train auc score = {np.mean(cv_scores_train)}")
    print(f"test auc score = {np.mean(cv_scores)}")
    mlflow.log_metric('train_auc', np.mean(cv_scores_train))
    mlflow.log_metric('test_auc', np.mean(cv_scores))
    return np.mean(cv_scores)

In [8]:
date_log = datetime.now().strftime("%Y_%m_%d_%H_%M")
study = optuna.create_study(direction="maximize", study_name=f"parameter_tuning_{config.country}_{date_log}")
func = lambda trial: objective(trial, df_pairs)
study.optimize(func, n_trials=100)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)


[I 2023-01-13 13:57:54,165] A new study created in memory with name: parameter_tuning_GBR_2023_01_13_13_57
Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.9920742	best: 0.9920742 (0)	total: 30.1ms	remaining: 5m 1s
100:	test: 0.9999019	best: 0.9999019 (100)	total: 1.13s	remaining: 1m 50s
200:	test: 0.9999135	best: 0.9999139 (193)	total: 2.24s	remaining: 1m 49s
bestTest = 0.9999167323
bestIteration = 242
Shrink model to first 243 iterations.


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.9926341	best: 0.9926341 (0)	total: 14.4ms	remaining: 2m 23s
100:	test: 0.9999115	best: 0.9999116 (98)	total: 1.15s	remaining: 1m 52s
200:	test: 0.9999218	best: 0.9999225 (197)	total: 2.29s	remaining: 1m 51s
bestTest = 0.9999258518
bestIteration = 247
Shrink model to first 248 iterations.


Default metric period is 5 because AUC is/are not implemented for GPU


...

0:	test: 0.9993418	best: 0.9993418 (0)	total: 168ms	remaining: 28m 1s
100:	test: 0.9999266	best: 0.9999266 (100)	total: 17.1s	remaining: 28m
200:	test: 0.9999327	best: 0.9999327 (187)	total: 33.9s	remaining: 27m 34s
300:	test: 0.9999348	best: 0.9999350 (291)	total: 50.8s	remaining: 27m 17s
bestTest = 0.9999358058
bestIteration = 356
Shrink model to first 357 iterations.


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.9994777	best: 0.9994777 (0)	total: 168ms	remaining: 28m 1s
100:	test: 0.9999284	best: 0.9999284 (100)	total: 17s	remaining: 27m 45s


[W 2023-01-13 16:42:50,848] Trial 57 failed because of the following error: KeyboardInterrupt('')
Traceback (most recent call last):
  File "/opt/conda/envs/rapids/lib/python3.8/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_3051/1914701348.py", line 3, in <lambda>
    func = lambda trial: objective(trial, df_pairs)
  File "/tmp/ipykernel_3051/3344502066.py", line 26, in objective
    model.fit(X_train[features],
  File "/opt/conda/envs/rapids/lib/python3.8/site-packages/catboost/core.py", line 5128, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "/opt/conda/envs/rapids/lib/python3.8/site-packages/catboost/core.py", line 2355, in _fit
    self._train(
  File "/opt/conda/envs/rapids/lib/python3.8/site-packages/catboost/core.py", line 1759, in _train
    self._object._train(train_pool, test_pool, params, all

KeyboardInterrupt: 

In [9]:
study.trials_dataframe().to_csv("/workspace/catboost_trials_v4.csv",index=None)